In [25]:
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [38]:
c.execute('''
SELECT name FROM sqlite_master WHERE type='table';
''')

conn.commit()
tables = c.fetchall()

In [41]:
[(table,) for table in tables]

[(('sqlite_sequence',),),
 (('Player_Attributes',),),
 (('Player',),),
 (('Match',),),
 (('League',),),
 (('Country',),),
 (('Team',),),
 (('Team_Attributes',),)]

In [96]:
def get_table_info(cursor, table):
    # Check column data types
    query = """
    pragma table_info({})
    """.format(table)
    cursor.execute(query)
    colnames = [col[1] for col in cursor.fetchall()]
    
    query = """
    SELECT * FROM {} LIMIT 5;
    """.format(table)
    

    table_first5 = pd.DataFrame(cursor.execute(query).fetchall(), columns=colnames)

    return table_first5

In [106]:
get_table_info(c, 'Match').iloc[:,1:20]

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9
0,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,None,None,None,None,None,None,None,None,None
1,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,None,None,None,None,None,None,None,None,None
2,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,None,None,None,None,None,None,None,None,None
3,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,None,None,None,None,None,None,None,None,None
4,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,None,None,None,None,None,None,None,None,None


In [99]:
get_table_info(c, 'Team')

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673,KRC Genk,GEN
1,2,9993,675,Beerschot AC,BAC
2,3,10000,15005,SV Zulte-Waregem,ZUL
3,4,9994,2007,Sporting Lokeren,LOK
4,5,9984,1750,KSV Cercle Brugge,CEB


### Q1: Which team scored the most points when playing at home?

In [113]:
query = '''
SELECT team_long_name, home_team_goal
FROM Team, Match
WHERE Team.team_api_id = Match.home_team_api_id
ORDER BY home_team_goal DESC
LIMIT 1;
'''

c.execute(query)
c.fetchone()

('PSV', 10)

### Q2: Did this team also score the most points when playing away?

In [114]:
query = '''
SELECT team_long_name, away_team_goal
FROM Team, Match
WHERE Team.team_api_id = Match.away_team_api_id
ORDER BY away_team_goal DESC
LIMIT 1;
'''

c.execute(query)
c.fetchone()

('Paris Saint-Germain', 9)

**This is not the same team which scores the most point when playing at home.**

### Q3: How many matches resulted in a tie?



In [126]:
query = '''
SELECT count(*)
FROM Match
WHERE home_team_goal = away_team_goal;
'''

c.execute(query)
tie_match = c.fetchone()[0]
print ('There are {} matches resulted in a tie'.format(tie_match))

There are 6596 matches resulted in a tie


### Q4: How many players have Smith for their last name? How many have 'smith' anywhere in their name?



In [117]:
get_table_info(c, 'Player')

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [121]:
query = '''
SELECT count(*)
FROM Player
WHERE player_name LIKE '%Smith';
'''

c.execute(query)
f_smith = c.fetchone()[0]
print ('There are {} players with Smith as family name.'.format(f_smith))

There are 18 players with Smith as family name.


In [125]:
query = '''
SELECT count(*)
FROM Player
WHERE player_name LIKE '%Smith%';
'''

c.execute(query)
any_smith = c.fetchone()[0]
print ('There are {} players with Smith in their names.'.format(any_smith))

There are 18 players with Smith in their names.


### Q5: What was the median tie score? Use the value determined in the previous question for the number of tie games. <br>
Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [161]:
query = '''
WITH TIE_SCORES(score) AS
(SELECT home_team_goal
FROM Match
WHERE home_team_goal = away_team_goal
ORDER BY home_team_goal DESC)
SELECT score
FROM TIE_SCORES
LIMIT 1
OFFSET (SELECT COUNT(*)
FROM TIE_SCORES)/2;
'''
c.execute(query)
print ('Median score for all the tied matches is {}'.format(c.fetchone()[0]))

Median score for all the tied matches is 1


### Q6: What percentage of players prefer their left or right foot? <br>
Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.


In [222]:
query = '''
SELECT (CAST(y.number AS FLOAT)/x.number)*100
FROM
(SELECT COUNT(*) as number
FROM Player_Attributes) x,
(SELECT COUNT(*) as number
FROM Player_Attributes
WHERE preferred_foot = 'right') y
'''
c.execute(query)
print ('{0:0.2f}% of the players prefer right foot.'.format(c.fetchone()[0]))

75.23% of the players prefer right foot.
